In [6]:
from dotenv import load_dotenv
import sys

load_dotenv()
sys.path.append("../")
from graph1 import graph

In [1]:
STR = """# MOOSE Input Card for Tensile Test on Rectangular Plate\n\n[Mesh]\n  type = GeneratedMesh\n  dim = 2\n  nx = 20  # Number of elements along the length\n  ny = 10  # Number of elements along the width\n  xmin = 0.0\n  xmax = 2.0  # Length of the plate\n  ymin = 0.0\n  ymax = 1.0  # Width of the plate\n  # Mesh refinement can be added here if needed\n[]\n\n[Variables]\n  [./disp_x]\n  [../]\n  [./disp_y]\n  [../]\n[]  # Define displacement variables for x and y directions\n\n[Kernels]\n  [SolidMechanics]\n    displacements = 'disp_x disp_y'  # Displacement variables\n  [../]\n[]  # Kernel for solid mechanics to compute displacements\n\n[BCs]\n  [./left_edge]\n    type = DirichletBC\n    variable = disp_x\n    boundary = 'left'\n    value = '0'  # Fixed boundary condition\n  [../]\n  [./right_edge]\n    type = TractionBC\n    variable = disp_y\n    boundary = 'right'\n    traction = '5e6'  # Traction boundary condition (5 MPa)\n  [../]\n[]  # Boundary conditions for fixed and traction\n\n[Materials]\n  [./material_properties]\n    type = ComputeIsotropicElasticityTensor\n    lambda = 200000  # Lame's first parameter (MPa)\n    shear_modulus = 76923.08  # Shear modulus derived from Young's modulus and Poisson's ratio\n  [../]\n[]  # Material properties for linear elasticity\n\n[Executioner]\n  type = Steady\n  solve_type = Newton\n  max_iterations = 100\n  tolerance = 1e-6  # Convergence criteria\n[]  # Execution settings for static analysis\n\n[Outputs]\n  file_base = tensile_test_results\n  exodus = false\n  [./csv]\n    type = CSV\n  [../]\n[]  # Output settings for results\n"""
print(STR)

# MOOSE Input Card for Tensile Test on Rectangular Plate

[Mesh]
  type = GeneratedMesh
  dim = 2
  nx = 20  # Number of elements along the length
  ny = 10  # Number of elements along the width
  xmin = 0.0
  xmax = 2.0  # Length of the plate
  ymin = 0.0
  ymax = 1.0  # Width of the plate
  # Mesh refinement can be added here if needed
[]

[Variables]
  [./disp_x]
  [../]
  [./disp_y]
  [../]
[]  # Define displacement variables for x and y directions

[Kernels]
  [SolidMechanics]
    displacements = 'disp_x disp_y'  # Displacement variables
  [../]
[]  # Kernel for solid mechanics to compute displacements

[BCs]
  [./left_edge]
    type = DirichletBC
    variable = disp_x
    boundary = 'left'
    value = '0'  # Fixed boundary condition
  [../]
  [./right_edge]
    type = TractionBC
    variable = disp_y
    boundary = 'right'
    traction = '5e6'  # Traction boundary condition (5 MPa)
  [../]
[]  # Boundary conditions for fixed and traction

[Materials]
  [./material_properties]
   

In [3]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
print(prompt)

/home/zt/miniforge/envs/langgraph/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [7]:
from IPython.display import Image, display

display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [15]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage


class AlignmentState(BaseModel):
    detailed_description: str = Field(description="what you want to say")


# 如果你的第三方大模型兼容 OpenAI Completion 接口，可以这样用:
llm = ChatOpenAI(
    model_name="Qwen/QwQ-32B",  # 或者换成你对应的模型
    temperature=0.7,
    base_url="https://api.siliconflow.cn/v1",
    api_key="sk-isdgjofhetjtwirzjjcrbaidzsonydgkmzfdqquymflnuaxh",
)
llm = llm.with_structured_output(AlignmentState)
res = llm.invoke(
    [
        SystemMessage(content="who you are?"),
    ]
)
res.detailed_description

'I am Qwen, a large language model developed by Alibaba Cloud. I have the ability to answer questions, create text such as stories, emails, scripts, and more. I can also perform logical reasoning, programming, express opinions, and play games. My capabilities are designed to assist users in a variety of tasks and provide engaging, informative responses.'

In [5]:
import re


def extract_sub_tasks(text):
    """
    从指定格式的文本中提取子任务信息。

    参数:
        text (str): 输入的文本，格式为：
                    **sub_task1:**
                    **Description:** detailed description of the sub-task.

    返回:
        list: 包含子任务信息的列表，每个子任务是一个字典。
    """
    # 使用正则表达式匹配子任务
    pattern = r"\*\*(.*?)\:\*\*\n\*\*Retrieve:\*\*(.*?)\n\*\*Description:\*\* (.*?)(?=\n\*\*|\Z)"
    matches = re.findall(pattern, text, re.DOTALL)

    # 将匹配结果存储到列表中
    result = [
        {"sub_task": sub_task.strip(), "retrieve": retrieve.strip(), "description": description.strip()}
        for sub_task, retrieve, description in matches
    ]

    return result


# 示例输入
input_text = """
**1:**
**Retrieve:**True
**Description:** detailed description of the sub-task.

**sub_task2:**
**Retrieve:**False
**Description:** detailed description of the sub-task.
"""

# 调用函数并打印结果
extracted_data = extract_sub_tasks(input_text)
for item in extracted_data:
    print(item)

{'sub_task': '1', 'retrieve': 'True', 'description': 'detailed description of the sub-task.'}
{'sub_task': 'sub_task2', 'retrieve': 'False', 'description': 'detailed description of the sub-task.'}
